In [1]:
import pandas as pd
import logging
import numpy as np
from datetime import datetime, timedelta
from kiteconnect import KiteConnect

#import apikey.py from Brokers folder


# Initialise Kite
acctkn_file = r'C:\Users\user\Desktop\GroundUp_Trading\NiftyStrategy\Brokers\acc_token.txt'
reqtkn_file = r'C:\Users\user\Desktop\GroundUp_Trading\NiftyStrategy\Brokers\req_token.txt'
kite_access_token = open(acctkn_file,'r').read()
kite_request_token = open(reqtkn_file,'r').read()
print(kite_access_token)

# Load the data
df = pd.read_csv(r'C:\Users\user\Desktop\GroundUp_Trading\NiftyStrategy\nf_backtest_consolidated.csv')
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%m/%d/%Y %H:%M')

df.set_index('DateTime', inplace=True)


dFS0buOe4dBTbK3AdfV2i1mG5C2uRBcc


In [2]:
expiry_date = '2023-05-25'
tokens = [256265]
trading_symbols = []

from_date = datetime.today().date()
to_date = datetime.today().date()
interval = 'minute'
qty = 50



In [3]:
def get_tokens(strike_prc,trend):
    global tokens,trading_symbols,expiry_date
    instruments_df = pd.read_csv(r'C:\Users\user\Desktop\GroundUp_Trading\NiftyStrategy\Brokers\instruments.csv')
    instruments_df = instruments_df[
    ["instrument_token", "tradingsymbol", "name", "exchange", "lot_size","instrument_type","expiry","strike"]
    ]
    
    if trend == "Long":
        option_type = "PE"
    elif trend == "Short":
        option_type = "CE"

    # from instruments dataframe, get all instruments for 'NFO' exchange, name =='NIFTY',expiry date is 4thMay2023, strike value between 17000 to 19000
    nfo_ins_df = instruments_df[
        (instruments_df["exchange"] == "NFO")
        & (instruments_df["name"] == "NIFTY")
        & (instruments_df["expiry"] == expiry_date)
        & (instruments_df["strike"] == strike_prc)
        &(instruments_df["instrument_type"] == option_type)
    ]
    
    tokens.append(nfo_ins_df['instrument_token'].values[0])
    trading_symbols.append(nfo_ins_df['tradingsymbol'].values[0])
      
    token = nfo_ins_df['tradingsymbol'].values[0]
    #get the third character from last to eighth character from last from the token
    digits = token[-7:-2]
    print(digits)
  
    option_type = token[-2:]
    print(option_type)
    
    if option_type == 'CE':
        new_numeric_part = int(digits) + 500
    elif option_type == 'PE':
        new_numeric_part = int(digits) - 500
    else:
        print('Error in option type')
        
    hedge_symbol = token.replace(str(digits), str(new_numeric_part))
    trading_symbols.append(hedge_symbol)
    
    return trading_symbols

In [4]:
kite = KiteConnect('6b0dp5ussukmo67h')
kite.set_access_token(kite_access_token)

In [5]:
nf_hist_data = kite.historical_data(tokens[0], from_date, to_date, interval)

# Convert historical_data to pandas dataframe
df = pd.DataFrame(nf_hist_data)
df = df.set_index('date')



In [20]:
def place_order(trading_symbol,trade_type):
    global qty
    kite = KiteConnect('6b0dp5ussukmo67h')
    kite.set_access_token(kite_access_token)
    if trade_type == 'BUY':
        order_type = kite.TRANSACTION_TYPE_BUY
    elif trade_type == 'SELL':
        order_type = kite.TRANSACTION_TYPE_SELL
    else:
        logging.info("Invalid trade type.")
    print(order_type)
    try:
        print("Placing order...")
        print(trading_symbol,order_type,qty )        
              
        
        order_id = kite.place_order(
            variety=kite.VARIETY_REGULAR,
            exchange=kite.EXCHANGE_NFO,
            tradingsymbol=trading_symbol,
            transaction_type=kite.TRANSACTION_TYPE_BUY,
            quantity=qty,
            product=kite.PRODUCT_NRML,
            order_type=kite.ORDER_TYPE_MARKET
    )
  
        logging.info("Order placed. ID is: {}".format(order_id))
    except Exception as e:
        logging.info("Order placement failed: {}".format(e))
    

In [24]:
nf_df = df.copy()

spot_prc = nf_df['close'].iloc[-1]
strike_prc = int(round(spot_prc/100,0)*100)

# Calculate the 15 min weighted average and the trend
previous_day_weighted_average = np.average(nf_df.between_time('14:30', '15:15')['close'], 
                                            weights=range(1, len(nf_df.between_time('14:30', '15:15')) + 1))
print(f"Weighted average for previous day: {previous_day_weighted_average}")

trend = 'Long' if nf_df.at_time('15:15')['close'].values[0] - previous_day_weighted_average > 0 else 'Short'
print(f"Trend: {trend}")

symbols = get_tokens(strike_prc,trend)
print(symbols)


Weighted average for previous day: 18280.20582793709
Trend: Long
18300
PE
['NIFTY23MAY18300PE', 'NIFTY23MAY17800PE', 'NIFTY23MAY18300PE', 'NIFTY23MAY17800PE', 'NIFTY23MAY18300PE', 'NIFTY23MAY17800PE', 'NIFTY23MAY18300PE', 'NIFTY23MAY17800PE', 'NIFTY23MAY18300PE', 'NIFTY23MAY17800PE', 'NIFTY23MAY18300PE', 'NIFTY23MAY17800PE', 'NIFTY23MAY18300PE', 'NIFTY23MAY17800PE']


InputException: Markets are closed right now. Use GTT for placing long standing orders instead. [Read more.](https://support.zerodha.com/category/trading-and-markets/gtt/articles/what-is-the-good-till-triggered-gtt-feature)

In [22]:
# Place orders
if trend == 'Long':
    place_order(symbols[0],'BUY')
    place_order(symbols[1],'SELL')
elif trend == 'Short':
    place_order(symbols[0],'SELL')
    place_order(symbols[1],'BUY')

BUY
Placing order...
NIFTY23MAY18300PE BUY 50
SELL
Placing order...
NIFTY23MAY17800PE SELL 50
